In [1]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/src')

In [3]:
# Install necessary packages from requirements.txt
req_path = '/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/requirements.txt'
with open(req_path, 'r') as file:
    requirements = file.read().splitlines()

for package in requirements:
    !pip install {package}

In [4]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [5]:
from google.colab import output
output.disable_custom_widget_manager()

In [6]:
from ragpipelines_pmi import *

In [7]:
# Load configuration from config.yaml
import yaml
config_path = '/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/config.yaml'
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

In [8]:
# Import libraries for evaluation
import pandas as pd
import numpy as np

In [9]:
from trulens.core import TruSession, Feedback

In [10]:
from trulens_eval import OpenAI, TruLlama

/tmp/ipykernel_2105/2882760375.py:1: DeprecationWarning: The `trulens_eval` module is deprecated. See https://www.trulens.org/trulens/guides/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval import OpenAI, TruLlama


In [11]:
from trulens_eval.feedback import GroundTruthAgreement

/tmp/ipykernel_2105/2870994414.py:1: DeprecationWarning: The `trulens_eval.feedback` module is deprecated. See https://www.trulens.org/trulens/guides/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval.feedback import GroundTruthAgreement


In [12]:
import os
os.environ['OPENAI_API_KEY'] = config['openai_api_key']

In [13]:
pip install "trulens-providers-openai>=1.0.0

/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [14]:
from trulens.core.session import TruSession

In [15]:
# Initialize TruLens and OpenAI provider
tru = TruSession()

🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


In [16]:
from trulens.providers.openai import OpenAI

In [17]:
openai_provider = OpenAI(model_engine="gpt-4o-mini")

In [18]:
tru.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [19]:
# Load evaluation questions and answers
eval_path = '/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/eval_questions_pmi.xlsx'
qa_df = pd.read_excel(eval_path, sheet_name="Sheet1")
qa_set = [{"query": item["Question"], "response": item["Answer"]} for index, item in qa_df.iterrows()]

In [20]:
from trulens.core.feedback.feedback import Feedback

In [21]:
# Define the evaluation metrics
# answer relevance
f_qa_relevance = Feedback(
    openai_provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [22]:
# context relevance
#f_qs_relevance = Feedback(
   # openai_provider.qs_relevance_with_cot_reasons, name="Context Relevance"
#).on_input().on(TruLlama.select_source_nodes().node.text).aggregate(np.mean)

In [23]:
from trulens_eval import Select

In [24]:
f_qs_relevance = (
    Feedback(
        openai_provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(Select.RecordCalls.retrieve.rets[:])
    .aggregate(np.mean)  # choose a different aggregation method if you wish
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets[:] .


In [25]:
# Groundedness (sometimes called faithfulness)
f_groundedness = (
    Feedback(openai_provider.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [26]:
# Ground truth agreement (sometimes called answer correctness)
f_groundtruth = Feedback(
    GroundTruthAgreement(qa_set).agreement_measure, name = "Answer Correctness"
).on_input_output()

✅ In Answer Correctness, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Correctness, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [27]:
ground_truth_collection = GroundTruthAgreement(qa_set, provider=openai_provider)

f_groundtruth = Feedback(ground_truth_collection.agreement_measure, name = "Answer Correctness").on_input_output()

✅ In Answer Correctness, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Correctness, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [28]:
metrics = [f_qa_relevance, f_qs_relevance, f_groundedness, f_groundtruth]

In [29]:
# Function to create TruLens recorder
def get_trulens_recorder(query_engine, app_id):
    return TruLlama(query_engine, feedbacks=metrics, app_id=app_id)

In [30]:
# Base class for RAG pipelines evaluation
class RAGEvaluator:
    def __init__(self, pipeline_class, app_id, api_key, document_path, llm_model, embed_model):
        self.pipeline = pipeline_class(api_key, document_path, llm_model, embed_model)
        self.query_engine = self.pipeline.query_engine
        self.recorder = get_trulens_recorder(self.query_engine, app_id=app_id)

    def evaluate(self, qa_set):
        with self.recorder as recording:
            for q in qa_set:
                self.query_engine.query(q["query"])


In [31]:
api_key = config['openai_api_key']
document_path = '/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/privacy-pmi.pdf'
llm_model = config['llm_model']
embed_model = config['embed_model']

In [32]:
# This function evaluates a given RAG pipeline by using the TruLens framework
def evaluate_pipeline(pipeline, qa_set, app_id, tru, metrics):
    # Initialize TruLens recorder for the pipeline
    recorder = TruLlama(
        app=pipeline.query_engine,  # Assuming the query_engine is the app here
        feedbacks=metrics,
        app_id=app_id
    )

    with recorder as recording:
        for q in qa_set:
            pipeline.query(q["query"])

    # Retrieve and return the records and feedback
    records, feedback = tru.get_records_and_feedback(app_ids=[app_id])
    return records, feedback

In [33]:
# Initialize and evaluate each pipeline

basic_pipeline = BasicRAGPipeline(api_key=config['openai_api_key'], document_path=document_path,
                                  llm_model=config['llm_model'], embed_model=config['embed_model'])

2024-10-15 19:22:44.146635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 19:22:44.228715: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 19:22:44.255720: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 19:22:44.316699: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 19:22:47.375919: W tensorflow/compiler/tf2

In [34]:
evaluate_pipeline(basic_pipeline, qa_set, "0. Basic Query Engine", tru, metrics)

(Empty DataFrame
 Columns: [app_id, app_json, type, record_id, input, output, tags, record_json, cost_json, perf_json, ts, latency, total_tokens, total_cost, cost_currency, app_name, app_version]
 Index: [],
 [])

In [35]:
sentence_window_rag_with_rerank = SentenceWindowRAGPipeline(api_key=config['openai_api_key'], document_path=document_path,
                                                            llm_model=config['llm_model'], embed_model=config['embed_model'],
                                                            rerank=True)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [36]:
evaluate_pipeline(sentence_window_rag_with_rerank, qa_set, "2. Sentence Window Query Engine - w/ re-ranking", tru, metrics)

(Empty DataFrame
 Columns: [app_id, app_json, type, record_id, input, output, tags, record_json, cost_json, perf_json, ts, latency, total_tokens, total_cost, cost_currency, app_name, app_version]
 Index: [],
 [])

In [37]:
sentence_window_rag_without_rerank = SentenceWindowRAGPipeline(api_key=config['openai_api_key'], document_path=document_path,
                                                               llm_model=config['llm_model'], embed_model=config['embed_model'],
                                                               rerank=False)

In [38]:
evaluate_pipeline(sentence_window_rag_without_rerank, qa_set, "1. Sentence Window Query Engine - W/o Reranking", tru, metrics)

(Empty DataFrame
 Columns: [app_id, app_json, type, record_id, input, output, tags, record_json, cost_json, perf_json, ts, latency, total_tokens, total_cost, cost_currency, app_name, app_version]
 Index: [],
 [])

In [39]:
automerging_pipeline = AutoMergingRAGPipeline(api_key=config['openai_api_key'], document_path=document_path,
                                              llm_model=config['llm_model'], embed_model=config['embed_model'])

In [40]:
evaluate_pipeline(automerging_pipeline, qa_set, "3. Automerging Query Engine - w/ re-ranking", tru, metrics)

> Merging 5 nodes into parent node.
> Parent node id: 5017c912-b5e4-4ed4-a2f1-c70bc7d18f21.
> Parent node text: Privacy Notice  of Philip Morris International  
 
Who are we?  
We are a member of Philip Morris...

> Merging 5 nodes into parent node.
> Parent node id: 5017c912-b5e4-4ed4-a2f1-c70bc7d18f21.
> Parent node text: Privacy Notice  of Philip Morris International  
 
Who are we?  
We are a member of Philip Morris...

> Merging 5 nodes into parent node.
> Parent node id: e2f2f2be-7bd2-4d7c-94aa-2b7dafece499.
> Parent node text: • Information about you will be shared with Philip Morris Products S.A.  (based in Neuchâtel, Swi...

> Merging 1 nodes into parent node.
> Parent node id: c103e358-6a23-49d6-a8b2-d380d0c6bb1f.
> Parent node text: - 
Right in respect of the information about you that we hold : to lodge a complaint with the sup...



(Empty DataFrame
 Columns: [app_id, app_json, type, record_id, input, output, tags, record_json, cost_json, perf_json, ts, latency, total_tokens, total_cost, cost_currency, app_name, app_version]
 Index: [],
 [])

In [41]:
automerging_pipeline_with_prompting = AutoMergingRAGPipelineWithPrompting(
    api_key=config['openai_api_key'],
    document_path=document_path,
    llm_model=config['llm_model'],
    embed_model=config['embed_model']
)

In [42]:
evaluate_pipeline(
    automerging_pipeline_with_prompting,
    qa_set,
    "4. Auto-Merging Query Engine - w/ re-ranking and prompt engineering",
    tru,
    metrics
)

> Merging 5 nodes into parent node.
> Parent node id: 5017c912-b5e4-4ed4-a2f1-c70bc7d18f21.
> Parent node text: Privacy Notice  of Philip Morris International  
 
Who are we?  
We are a member of Philip Morris...



/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/src/ragpipelines_pmi.py:519: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.llm_langchain(full_prompt)


> Merging 5 nodes into parent node.
> Parent node id: 5017c912-b5e4-4ed4-a2f1-c70bc7d18f21.
> Parent node text: Privacy Notice  of Philip Morris International  
 
Who are we?  
We are a member of Philip Morris...

> Merging 5 nodes into parent node.
> Parent node id: e2f2f2be-7bd2-4d7c-94aa-2b7dafece499.
> Parent node text: • Information about you will be shared with Philip Morris Products S.A.  (based in Neuchâtel, Swi...

> Merging 1 nodes into parent node.
> Parent node id: c103e358-6a23-49d6-a8b2-d380d0c6bb1f.
> Parent node text: - 
Right in respect of the information about you that we hold : to lodge a complaint with the sup...



(Empty DataFrame
 Columns: [app_id, app_json, type, record_id, input, output, tags, record_json, cost_json, perf_json, ts, latency, total_tokens, total_cost, cost_currency, app_name, app_version]
 Index: [],
 [])

In [43]:
# launches on http://localhost:8501/
tru.run_dashboard()

/tmp/ipykernel_2105/1997498841.py:2: DeprecationWarning: Method `run_dashboard` has been renamed or moved to `trulens.dashboard.run.run_dashboard`.

  tru.run_dashboard()


Starting dashboard ...
npm WARN exec The following package was not found and will be installed: localtunnel@2.0.2

Go to this url and submit the ip given here. your url is: https://chilly-cycles-attack.loca.lt

  Submit this IP Address: 34.83.73.65



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
#tru.stop_dashboard()